In [37]:
# from pytickersymbols import PyTickerSymbolsL
import yfinance as yf
import pandas as pd
import numpy as np
import os
import json

import time

In [21]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
DATA_PATH = './data'
os.makedirs(DATA_PATH, exist_ok=True)

# 1. Get all funds listed on the US NASDAQ exchange

In [15]:
TOTAL_FUNDS = 46861
PAGE_SIZE = 250
EXCHANGE = 'NAS'

In [16]:
def fetch_funds(exchange=EXCHANGE, page_size=PAGE_SIZE, total=TOTAL_FUNDS):
    # build query
    q = yf.FundQuery('eq', ['exchange', exchange])
    all_results = []

    offset = 0

    while True:
        print(f"Fetching offset {offset} ...")
        # Fetch one page
        page = yf.screen(q, size=page_size, offset=offset)
        # Safety stop: if empty page, stop
        if not page or len(page['quotes']) == 0:
            print("No more results returned → stopping.")
            break
        all_results.extend(page['quotes'])
        # Update offset for next page
        offset += page_size
        # Optional: stop early if we passed expected total
        if offset >= total:
            print("Reached total expected count. Finished.")
            break
        # Optional: avoid request throttling
        time.sleep(0.1)
    return all_results

In [17]:
# funds_results = fetch_funds()

Fetching offset 0 ...
Fetching offset 250 ...
Fetching offset 500 ...
Fetching offset 750 ...
Fetching offset 1000 ...
Fetching offset 1250 ...
Fetching offset 1500 ...
Fetching offset 1750 ...
Fetching offset 2000 ...
Fetching offset 2250 ...
Fetching offset 2500 ...
Fetching offset 2750 ...
Fetching offset 3000 ...
Fetching offset 3250 ...
Fetching offset 3500 ...
Fetching offset 3750 ...
Fetching offset 4000 ...
Fetching offset 4250 ...
Fetching offset 4500 ...
Fetching offset 4750 ...
Fetching offset 5000 ...
Fetching offset 5250 ...
Fetching offset 5500 ...
Fetching offset 5750 ...
Fetching offset 6000 ...
Fetching offset 6250 ...
Fetching offset 6500 ...
Fetching offset 6750 ...
Fetching offset 7000 ...
Fetching offset 7250 ...
Fetching offset 7500 ...
Fetching offset 7750 ...
Fetching offset 8000 ...
Fetching offset 8250 ...
Fetching offset 8500 ...
Fetching offset 8750 ...
Fetching offset 9000 ...
Fetching offset 9250 ...
Fetching offset 9500 ...
Fetching offset 9750 ...
Fetchi

In [19]:
# len(funds_results)

47000

In [21]:
# convert to table
funds_df = pd.DataFrame(funds_results)

In [23]:
funds_df[['symbol', 'regularMarketPrice']].head()

,symbol,regularMarketPrice
0,ZZZZIX,10.999
1,ZZZAX,1.000
2,ZZYYX,7.850
3,ZZWWX,24163.530
4,ZYXZZX,2.000


In [28]:
funds_df.to_parquet(os.path.join(DATA_PATH, 'nas_funds.parquet'), index=False)

# 2. Get all stocks listed on US exchanges

This should give us a list of stock symbols. To get the info, we'll use a different API endpoint (see section 3)

In [93]:
TOTAL_STOCKS = 10000
PAGE_SIZE = 250
EXCHANGE = [
    # "SHH", # China Shanghai
    # "JPX", # Japan (Tokyo + Osaka)
    # "KOE", # South Korea KOSPI main
    # "TAI", # Taiwan

    # "FRA", # Germany Frankfurt Xetra
    # "LSE", # UK

    "NMS", # NASDAQ main
    "NYQ", # NYSE main
    "ASE", # NYSE American (AMEX)
]

In [94]:
def fetch_stocks(exchange=EXCHANGE, page_size=PAGE_SIZE, total=TOTAL_STOCKS):
    # build query
    q = yf.EquityQuery('is-in', ['exchange', *exchange])
    all_results = []

    offset = 0

    while True:
        print(f"Fetching offset {offset} ...")
        # Fetch one page
        page = yf.screen(q, size=page_size, offset=offset)
        # Safety stop: if empty page, stop
        if not page or len(page['quotes']) == 0:
            print("No more results returned → stopping.")
            break
        all_results.extend(page['quotes'])
        # Update offset for next page
        offset += page_size
        # Optional: stop early if we passed expected total
        if offset >= total:
            print("Reached total expected count. Finished.")
            break
        # Optional: avoid request throttling
        time.sleep(0.1)
    return all_results

In [95]:
stocks_result = fetch_stocks()

Fetching offset 0 ...
Fetching offset 250 ...
Fetching offset 500 ...
Fetching offset 750 ...
Fetching offset 1000 ...
Fetching offset 1250 ...
Fetching offset 1500 ...
Fetching offset 1750 ...
Fetching offset 2000 ...
Fetching offset 2250 ...
Fetching offset 2500 ...
Fetching offset 2750 ...
Fetching offset 3000 ...
Fetching offset 3250 ...
Fetching offset 3500 ...
Fetching offset 3750 ...
Fetching offset 4000 ...
Fetching offset 4250 ...
Fetching offset 4500 ...
Fetching offset 4750 ...
No more results returned → stopping.


In [96]:
len(stocks_result)

4607

In [98]:
stocks_df = pd.DataFrame(stocks_result)

In [99]:
stocks_df.to_parquet(os.path.join(DATA_PATH, 'stocks.parquet'), index=False)

# 3. Get stock's info

This is what we're using to extract ticker's relevant info (display name, sector, C-level members, etc.)

In [5]:
stocks_df = pd.read_parquet(os.path.join(DATA_PATH, 'stocks.parquet'))

In [4]:
def fetch_stock_info(symbol):
    try:
        t = yf.Ticker(symbol)
        return t.info
    except:
        return {}

In [6]:
all_rows = []

for symbol in stocks_df['symbol']:
    try:
        info = fetch_stock_info(symbol)
        all_rows.append({
            'symbol': symbol,
            'info': info,
        })
        
        # Sleep to avoid API throttling
        time.sleep(0.05)
    
    except Exception as e:
        all_rows.append({
            'symbol': symbol,
            'info': {},
        })

In [11]:
all_rows = [r for r in all_rows if len(r['info'].keys()) > 0]

In [12]:
len(all_rows)

4607

In [13]:
# convert to dataframe
info_df = pd.DataFrame(all_rows)

info_df.head()

,symbol,info
0,ZYXI,"{'address1': '9655 Maroon Circle', 'city': 'En..."
1,ZYME,"{'address1': '108 Patriot Drive', 'address2': ..."
2,ZWS,"{'address1': '511 West Freshwater Way', 'city'..."
3,ZVRA,"{'address1': '1180 Celebration Boulevard', 'ad..."
4,ZVIA,"{'address1': '15821 Ventura Boulevard', 'addre..."


In [31]:
df_final = pd.json_normalize(info_df['info'])

In [35]:
df_final.dtypes

address1                              object
city                                  object
state                                 object
zip                                   object
country                               object
phone                                 object
fax                                   object
website                               object
industry                              object
industryKey                           object
industryDisp                          object
sector                                object
sectorKey                             object
sectorDisp                            object
longBusinessSummary                   object
fullTimeEmployees                    float64
companyOfficers                       object
compensationAsOfEpochDate            float64
executiveTeam                         object
maxAge                                 int64
priceHint                              int64
previousClose                        float64
open      

In [40]:
# replace Infinity values with np.inf
df_final[['priceToSalesTrailing12Months', 'trailingPE']] = df_final[['priceToSalesTrailing12Months', 'trailingPE']].replace('Infinity', np.inf)

/tmp/ipykernel_23789/1314993215.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final[['priceToSalesTrailing12Months', 'trailingPE']] = df_final[['priceToSalesTrailing12Months', 'trailingPE']].replace('Infinity', np.inf)


In [41]:
df_final.to_parquet(os.path.join(DATA_PATH, "stocks_info.parquet"))

# 4. Get stocks' institutional holders

In [116]:
stocks_df = pd.read_parquet(os.path.join(DATA_PATH, 'stocks.parquet'))

# stocks_df.head()

,symbol
0,ZYXI
1,ZYME
2,ZWS
3,ZVRA
4,ZVIA


In [113]:
def get_institutional_holders(symbol):
    try:
        t = yf.Ticker(symbol)
        # Get institutional holders as DataFrame
        holders = t.get_institutional_holders(as_dict=False).to_dict(orient='records')
    except:
        holders = []
    return holders

In [117]:
all_rows = []

for symbol in stocks_df['symbol']:
    try:
        holders = get_institutional_holders(symbol)
        all_rows.append({
            'symbol': symbol,
            'holders': holders,
        })
        
        # Sleep to avoid API throttling
        time.sleep(0.1)
    
    except Exception as e:
        all_rows.append({
            'symbol': symbol,
            'holders': [],
        })

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: XRPM"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: VUSV"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: VUSG"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: VDIG"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: TSOL"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: TEST"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: NVIT"}}}
HTTP Error 404: {"quoteSummary":{"

In [118]:
# convert to dataframe
holders_df = pd.DataFrame(all_rows)

holders_df.head()

,symbol,holders
0,ZYXI,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."
1,ZYME,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."
2,ZWS,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."
3,ZVRA,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."
4,ZVIA,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."


In [5]:
holders_df.to_parquet(os.path.join(DATA_PATH, "institutional_holders.parquet"))

# 5. Get stocks' mutual fund holders

In [12]:
stocks_df = pd.read_parquet(os.path.join(DATA_PATH, 'stocks.parquet'))

stocks_df['symbol'].head()

0    ZYXI
1    ZYME
2     ZWS
3    ZVRA
4    ZVIA
Name: symbol, dtype: object

In [13]:
def get_mutualfund_holders(symbol):
    try:
        t = yf.Ticker(symbol)
        # Get institutional holders as DataFrame
        holders = t.get_mutualfund_holders(as_dict=False).to_dict(orient='records')
    except:
        holders = []
    return holders

In [31]:
all_rows = []

for symbol in stocks_df['symbol']:
    try:
        holders = get_mutualfund_holders(symbol)
        all_rows.append({
            'symbol': symbol,
            'holders': holders,
        })
        
        # Sleep to avoid API throttling
        time.sleep(0.05)
    
    except Exception as e:
        all_rows.append({
            'symbol': symbol,
            'holders': [],
        })

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: XRPM"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: VUSV"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: VUSG"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: VDIG"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: TSOL"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: TEST"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"No fundamentals data found for symbol: NVIT"}}}
HTTP Error 404: {"quoteSummary":{"

In [34]:
# convert to dataframe
holders_df = pd.DataFrame(all_rows)

holders_df.head()

,symbol,holders
0,ZYXI,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."
1,ZYME,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."
2,ZWS,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."
3,ZVRA,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."
4,ZVIA,"[{'Date Reported': 2025-09-30 00:00:00, 'Holde..."


In [33]:
holders_df.to_parquet(os.path.join(DATA_PATH, "mutualfund_holders.parquet"))

# 6. Get stock's related tickers in latest 10 pieces of news featuring it

In [42]:
stocks_df = pd.read_parquet(os.path.join(DATA_PATH, 'stocks.parquet'))

stocks_df['symbol'].head()

0    ZYXI
1    ZYME
2     ZWS
3    ZVRA
4    ZVIA
Name: symbol, dtype: object

In [43]:
def get_related_tickers(symbol, news_count=10):
    try:
        search = yf.Search(symbol, news_count=news_count)
        news = [n.get('relatedTickers', []) for n in search.news]
        related_tickers = list(set({item for sublist in news for item in sublist}))
    except:
        related_tickers = []
    return related_tickers

In [44]:
all_rows = []

for symbol in stocks_df['symbol']:
    try:
        related = get_related_tickers(symbol)
        all_rows.append({
            'symbol': symbol,
            'related_tickers': related,
        })
        
        # Sleep to avoid API throttling
        time.sleep(0.05)
    
    except Exception as e:
        all_rows.append({
            'symbol': symbol,
            'related_tickers': [],
        })

In [45]:
# convert to dataframe
related_df = pd.DataFrame(all_rows)

related_df.head()

,symbol,related_tickers
0,ZYXI,"[ESSITY-B.ST, CLSK, NVDA, BITF, ASST, IBIT, XR..."
1,ZYME,"[JAZZ, ZYME]"
2,ZWS,"[ZGSB.TO, ZWB.TO, ZGLD.TO, ZCDB.TO, ZHU.TO, ZM..."
3,ZVRA,"[XOMA, EPIX, ZVRA, MURA, TSBX, XOMAP, XOMAO, L..."
4,ZVIA,"[UPLD, ^SPX, WMT, BYND, PLTR, ZVIA, KHC, PRKS,..."


In [46]:
related_df.to_parquet(os.path.join(DATA_PATH, "stocks_related_tickers.parquet"))